<a href="https://colab.research.google.com/github/batust/Pre-processing/blob/main/pre-prcoessor-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Tranformers library
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = """My name is Allwyn Bat Thomas and I live in Kothamangalam. My phone number is 0509821676"""

ner_results = nlp(example)
print(ner_results)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'entity': 'B-PER', 'score': 0.99717784, 'index': 4, 'word': 'All', 'start': 11, 'end': 14}, {'entity': 'I-PER', 'score': 0.5963278, 'index': 5, 'word': '##wyn', 'start': 14, 'end': 17}, {'entity': 'I-PER', 'score': 0.9789915, 'index': 6, 'word': 'Bat', 'start': 18, 'end': 21}, {'entity': 'I-PER', 'score': 0.9963566, 'index': 7, 'word': 'Thomas', 'start': 22, 'end': 28}, {'entity': 'B-LOC', 'score': 0.997614, 'index': 12, 'word': 'Ko', 'start': 43, 'end': 45}, {'entity': 'I-LOC', 'score': 0.97299033, 'index': 13, 'word': '##tham', 'start': 45, 'end': 49}, {'entity': 'I-LOC', 'score': 0.99213374, 'index': 14, 'word': '##anga', 'start': 49, 'end': 53}, {'entity': 'I-LOC', 'score': 0.996071, 'index': 15, 'word': '##lam', 'start': 53, 'end': 56}]


In [ ]:
# Anonymizing data from transformer side
def anonymize(text, results):
  entities = {}
  anonymized_text = text
  prev_start = None

  for entity in results:
    entity_type = entity['entity'].split('-')[-1]
    word = entity['word'].replace('##', '')
    start, end = entity['start'], entity['end']

    if prev_start is not None and entities[prev_start]['end'] == start:
      entities[prev_start]['word'] += word
      entities[prev_start]['end'] = end

    else:
      entities[start] = {'word':word, 'end': end, 'entity_type': entity_type}
      prev_start = start

  for item in entities.items():
    print('Iter:', item)

anonymize(example, ner_results)


Iter: (11, {'word': 'Allwyn', 'end': 17, 'entity_type': 'PER'})
Iter: (18, {'word': 'Bat', 'end': 21, 'entity_type': 'PER'})
Iter: (22, {'word': 'Thomas', 'end': 28, 'entity_type': 'PER'})
Iter: (43, {'word': 'Kothamangalam', 'end': 56, 'entity_type': 'LOC'})


In [ ]:
# Creating patterns in presidio
from presidio_analyzer import PatternRecognizer, AnalyzerEngine

titles_list = [
    "Sir",
    "Ma'am",
    "Madam",
    "Mr.",
    "Mrs.",
    "Ms.",
    "Miss",
    "Dr.",
    "Professor",
]

text = 'Sir Allwyn Bat Thomas is a CIA agent!'

pattern = PatternRecognizer(
    supported_entity="TITLE",
    deny_list=titles_list,
)

result = pattern.analyze(
    text=text,
    entities=['TITLE']
)
print(f'Result: {result}')

analyzer = AnalyzerEngine()
analyzer.registry.add_recognizer(pattern)
ans = analyzer.analyze(text=text, language='en')
print(ans)

Result: [type: TITLE, start: 0, end: 3, score: 1.0]


[type: TITLE, start: 0, end: 3, score: 1.0, type: PERSON, start: 4, end: 21, score: 0.85]


In [ ]:
json_data = {
    "name": "Allwyn Bat Thomas",
    "email":[ "allwynbatthomas@ust.com", 'allwynt700@gmail.com', '287945@ust.com'],
    "backup_email": "b20cs100@mace.ac.in",
    "phone": "+973 30982167",
    "location": "Wakanda, Africa",
    "description": "Allwyn is a software engineer from Wakanda.",
    "metadata": {
        "emergency_contact": "+91 9678785654",
        "company": "UST Global",
        "address": {
            "city": "New York",
            "country": "USA"
        }
    }
}

In [16]:
# Processing JSON Spacey
def processing_json(data):
  if isinstance(data, dict):
    return {key: processing_json(value) for key, value in data.items()}
  elif isinstance(data, list):
    return [processing_json(item) for item in data]
  elif isinstance(data, str):
    result = analyzer.analyze(
        text = data,
        entities = ['PHONE_NUMBER', 'ADDRESS', 'LOCATION', 'PERSON', 'EMAIL_ADDRESS', 'STREET_ADDRESS'],
        language = 'en'
    )
    return anonymizer.anonymize(text=data, analyzer_results=result).text if result else data
  else: return data

In [17]:
# Uploading json file and doing NER on attributes
import json
from google.colab import files
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

file_upload = files.upload()
for filename in file_upload.keys():
  if filename.endswith('.json'):
    with open(filename, 'r') as fp:
      json_data = json.load(fp)
      processed_data = processing_json(json_data)
      print(json.dumps(processed_data, indent = 4))

Saving test.json to test.json
{
    "name": "<PERSON>",
    "age": 25,
    "city": "<LOCATION>",
    "married": false,
    "hobbies": [
        "reading",
        "traveling",
        "programming"
    ],
    "address": {
        "street": "Civil Station, Kudappanakkunnu",
        "city": "<LOCATION>",
        "postal_code": "695043"
    },
    "country": "<LOCATION>",
    "phone": "<PHONE_NUMBER>"
}


In [ ]:
addresses = [
    '123 MG Road, Indiranagar, Bengaluru 560038, Karnataka, INDIA',
    'ul. Tverskaya, d. 16, kv. 5, Moskva 125009, RUSSIA',
    '123 Main Street, Springfield, IL 62701, USA',
    'Musterstraße 1, 12345 Musterstadt, GERMANY',
    'Al-Mashtal Street, Khartoum 11111, SUDAN',
    'King Abdulaziz Road, Building 15, Al-Murabba District, 11564 Riyadh, SAUDI ARABIA',
]

In [14]:
# Trying to detect addresses
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer import AnalyzerEngine, PatternRecognizer, Pattern

ADDRESS_REGEX = r"(\d*?[a-zA-z-]?,?)?\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b"
address_pattern = Pattern(
    name="address",
    regex=ADDRESS_REGEX,
    score=0.8
)
address_recognizer = PatternRecognizer(
    supported_entity="ADDRESS",
    patterns=[address_pattern]
)
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()
analyzer.registry.add_recognizer(address_recognizer)

for address in addresses:
  results = analyzer.analyze(
      text=address,
      entities=['ADDRESS', 'LOCATION', 'PERSON', 'EMAIL_ADDRESS', 'PHONE_NUMBER'],
      language='en'
  )
  if results:
    print(
        anonymizer.anonymize(
            text=address,
            analyzer_results=results
        ).text
    )


<ADDRESS>, <LOCATION>, <LOCATION> 560038, <LOCATION>, <LOCATION>
ul. <PERSON>, d. 16, kv. 5, Moskva 125009, <LOCATION>
<ADDRESS>, <LOCATION>, IL 62701, <LOCATION>
<PERSON>, 12345 <LOCATION>, <LOCATION>
Al-Mashtal Street, <LOCATION>, SUDAN
Kin<ADDRESS>, <LOCATION>, <PERSON>, 11564 <LOCATION>, SAUDI ARABIA


**TESTING OUT FLAIR**

In [1]:
!pip install flair
!pip install presidio_analyzer
!pip install presidio_anonymizer
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [41]:
story_text = '''My name is Goldman, my email is atrealgoldman@goldman.com'''

In [42]:
# Solution ( kinda ;) )
from flair.data import Sentence
from flair.nn import Classifier
from presidio_analyzer import AnalyzerEngine, RecognizerResult, Pattern, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from flair.models import SequenceTagger

# Presidio objects
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# Address pattern
ADDRESS_REGEX = r"\d+\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b"
street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=ADDRESS_REGEX,
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)

analyzer.registry.add_recognizer(street_recognizer)

def presidio_text(data):
  results = analyzer.analyze(
      text=data,
      entities=['PERSON', 'EMAIL_ADDRESS', 'PHONE_NUMBER', 'STREET_ADDRESS', 'LOCATION'],
      language='en'
  )
  return results

def flair_text(story_text):
  sentence = Sentence(story_text)
  tagger = SequenceTagger.load("flair/ner-english-large")
  tagger.predict(sentence)

  print('Printing:', tagger.predict(sentence))
  print('Printing:', sentence.to_tagged_string())

  entity_map = {
        'LOC': 'LOCATION',
        'GPE': 'LOCATION',
        'ORG': 'ORGANIZATION',
        'PER': 'PERSON'
    }

  return [
      RecognizerResult(
          entity_type=entity_map[entity.tag],
          start=entity.start_position,
          end=entity.end_position,
          score=entity.score,
      )
      for entity in sentence.get_spans('ner') if entity.tag in entity_map
  ]

flair_data = flair_text(story_text)
presidio_data = presidio_text(story_text)

print('Flair Data:', flair_data)
print('Presidio Data:', presidio_data)

all_data = presidio_data + flair_data

updated_text = anonymizer.anonymize(
    text=story_text,
    analyzer_results=all_data
).text

print('\n\n', updated_text)


2025-03-11 10:47:40,517 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Printing: None
Printing: Sentence[11]: "My name is Goldman, my email is atrealgoldman@goldman.com" → ["Goldman"/PER, "goldman.com"/ORG]
Flair Data: [type: PERSON, start: 11, end: 18, score: 0.9999889135360718, type: ORGANIZATION, start: 46, end: 57, score: 0.998198926448822]
Presidio Data: [type: EMAIL_ADDRESS, start: 32, end: 57, score: 1.0]


 My name is <PERSON>, my email is <EMAIL_ADDRESS>


In [8]:
json_test_data = {
    "name": "Allwyn BAT Thomas",
    "age": 25,
    "city": "Thiruvananthapuram",
    "married": False,
    "hobbies": ["reading", "traveling", "programming"],
    "address": {
        "street": "Civil Station, Kudappanakkunnu",
        "city": "Thiruvananthapuram City",
        "postal_code": "695043 Area Code"
    },
    "country": "Australia",
    "phone": "+61 8 8672 4617"
}

In [17]:
# Midhun

from presidio_analyzer import AnalyzerEngine, RecognizerResult, Pattern, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from flair.models import SequenceTagger
from flair.data import Sentence
from presidio_analyzer.nlp_engine import NlpEngineProvider

ner_tagger = Classifier.load('flair/ner-english-large')

nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}]
}
street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)



provider = NlpEngineProvider(nlp_configuration=nlp_config)
nlp_engine = provider.create_engine()

analyzer=AnalyzerEngine(nlp_engine=nlp_engine)
anonymizer=AnonymizerEngine()
analyzer.registry.add_recognizer(street_recognizer)

def detect_locations_with_flair(text):
    sentence = Sentence(text)
    ner_tagger.predict(sentence)

    location_entities = []
    for entity in sentence.get_spans("ner"):
        if entity.tag in ["LOC","ORG", 'PER']:
          entity_type = "LOCATION" if entity.tag == "LOC" else "ORGANIZATION" if entity.tag == "ORG" else "PERSON"
          location_entities.append(RecognizerResult(
                entity_type=entity_type,
                start=entity.start_position,
                end=entity.end_position,
                score=entity.score
            ))
    return location_entities

def analyze_and_anonymize(text):
    presidio_results = analyzer.analyze(
        text=text,
        entities=["EMAIL_ADDRESS", "PHONE_NUMBER", "STREET_ADDRESS"],
        language="en"
    )

    flair_results = detect_locations_with_flair(text)
    print(flair_results)
    print(presidio_results)
    all_results = presidio_results + flair_results

    anonymized_text = anonymizer.anonymize(text, all_results).text
    return anonymized_text

texts=" My name is Apple. "

anonymized_text = analyze_and_anonymize(texts)
print(anonymized_text)



2025-03-11 10:08:49,505 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


[type: PERSON, start: 12, end: 17, score: 0.9994992017745972]
[]
 My name is <PERSON>. 
